In [ ]:
import os
import json
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

# API configuration
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

# File paths
CLIENT_SECRETS_FILE = 'client_secret.json'  # Path to your client secrets
SUBSCRIPTIONS_FILE = 'subscriptions.json'  # Path to your subscriptions file

# Authenticate and get the YouTube API service
creds = None

# Always trigger OAuth flow and authenticate
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
creds = flow.run_local_server(port=0)  # This will handle the OAuth flow automatically

# Build the YouTube service
youtube_service = build(API_SERVICE_NAME, API_VERSION, credentials=creds)

# Process the subscriptions from JSON file to subscribe
try:
    with open(SUBSCRIPTIONS_FILE, 'r') as file:
        data = json.load(file)
        subscriptions = data.get('subscriptions', [])
        
        if not subscriptions:
            print("No subscriptions found in the file.")
        
        for channel_obj in subscriptions:
            url = channel_obj.get('url')
            name = channel_obj.get('name', 'Unknown Channel')
            
            if not url:
                print(f"Skipping entry with missing URL: {name}")
                continue
            
            # Extract channel ID from URL
            if '/channel/' in url:
                channel_id = url.split('/channel/')[-1].split('/')[0]
            else:
                print(f"Skipping invalid URL for {name}: {url}")
                continue
            
            # Create subscription request
            body = {
                'snippet': {
                    'resourceId': {
                        'kind': 'youtube#channel',
                        'channelId': channel_id
                    }
                }
            }
            
            try:
                response = youtube_service.subscriptions().insert(
                    part='snippet',
                    body=body
                ).execute()
                print(f"Subscribed to {name} ({response['snippet']['title']})")
            except HttpError as e:
                if e.resp.status == 400 and 'already exists' in str(e):
                    print(f"Already subscribed to {name} ({channel_id})")
                else:
                    print(f"Error subscribing to {name}: {str(e)}")

except FileNotFoundError:
    print("subscriptions.json file not found!")
except json.JSONDecodeError:
    print("Invalid JSON format in subscriptions.json")
except Exception as e:
    print(f"An error occurred: {str(e)}")